# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [6]:
weather_survey = pd.read_csv('../output_data/weather_survey.csv')
weather_survey.head()

,City,Latitude,Longitude,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date
0,Rikitea,-23.12,-134.97,292.54,63,0,7.55,PF,1595534596
1,Bluff,-46.60,168.33,277.04,83,98,1.34,NZ,1595534626
2,Thompson,55.74,-97.86,299.15,65,75,4.10,CA,1595534424
3,Kenai,60.55,-151.26,290.15,63,1,2.60,US,1595534805
4,Bredasdorp,-34.53,20.04,285.15,93,0,1.00,ZA,1595534616


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [25]:
gmaps.configure(api_key=g_key)

In [106]:
humidity_fig = gmaps.figure(center=(30,0), zoom_level=1.2)

locations = weather_survey[['Latitude','Longitude']]
humidity = weather_survey['Humidity']

humidity_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=10, point_radius=1)

humidity_fig.add_layer(humidity_layer)

humidity_fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [30]:
#Cloudy vacation spots with high humidity
cvswhh = weather_survey.loc[(weather_survey['Cloudiness']>75)&(weather_survey['Humidity']>75)].dropna()

cvswhh.head()

,City,Latitude,Longitude,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date
1,Bluff,-46.60,168.33,277.04,83,98,1.34,NZ,1595534626
8,Yellowknife,62.46,-114.35,289.26,87,90,6.20,CA,1595535186
16,Tiksi,71.69,128.87,287.97,81,98,3.76,RU,1595534636
17,Ballina,-28.87,153.57,288.15,93,90,2.10,AU,1595534847
18,City of San Pedro,14.35,121.02,304.26,88,100,2.60,PH,1595535188


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [81]:
hotel_df = cvswhh
hotel_names = []
hotels_lat = []
hotels_lng = []

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "rankby": "distance",
    "key": g_key,
    "types": "hotel",
    "keyword": 'hotel',
}

for index, row in hotel_df.iterrows():
    lat = row['Latitude']
    lng = row['Longitude']
    params['location'] = (f"{lat},{lng}")
    try:
        response = requests.get(base_url, params=params).json()        
        hotel_name = response['results'][0]['name']
        hotel_lat = response['results'][0]['geometry']['location']['lat']
        hotel_lng = response['results'][0]['geometry']['location']['lng']
        hotel_names.append(hotel_name)
        hotels_lat.append(hotel_lat)
        hotels_lng.append(hotel_lng)
        print(f"{hotel_name}: Latitude: {hotel_lat}; Longitude: {hotel_lng}")
    except (KeyError, IndexError):
        pass
        hotel_name = ''
        hotel_lat = ''
        hotel_lng = ''
        hotel_names.append(hotel_name)
        hotels_lat.append(hotel_lat)
        hotels_lng.append(hotel_lng)        

The Eagle: Latitude: -46.59895239999999; Longitude: 168.3417801
Mo's Houseboat B&B: Latitude: 62.45879489999999; Longitude: -114.3429329
Arktika: Latitude: 71.6922089; Longitude: 128.871611
Ballina Manor Boutique Hotel: Latitude: -28.8702686; Longitude: 153.567272
Palines Apartment and Guesthouse: Latitude: 14.364308; Longitude: 121.0128772
Airport Inn: Latitude: 71.2888583; Longitude: -156.779611
Gran Hotel Panamericano: Latitude: -37.99771; Longitude: -57.549979
Gostinitsa "Zapolyar'ye": Latitude: 71.9782487; Longitude: 102.4688095
The Fern & Thistle Accommodation: Latitude: -46.226144; Longitude: 169.7898766
Yeet Hotel: Latitude: 61.99411509999999; Longitude: -49.6665709
Hotel Santika Bengkulu: Latitude: -3.7978888; Longitude: 102.2729506
SMD Tiffin Parlour: Latitude: 18.7650325; Longitude: 84.4188463
Taloa Heights: Latitude: -19.0607788; Longitude: -169.9217526
Basawasri Lodge: Latitude: 16.7277581; Longitude: 75.0651985
Quality Hotel Bodensia: Latitude: 65.82467030000001; Longitud

In [104]:
hotel_df['Hotel_Name'] = hotel_names
hotel_df['Hotel_Lat'] = hotels_lat
hotel_df['Hotel_Lng'] = hotels_lng
hotel_df1 = hotel_df.loc[hotel_df['Hotel_Name'] !='']
hotel_df1.head()

,City,Latitude,Longitude,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date,Hotel_Name,Hotel_Lat,Hotel_Lng
1,Bluff,-46.60,168.33,277.04,83,98,1.34,NZ,1595534626,The Eagle,-46.599,168.342
8,Yellowknife,62.46,-114.35,289.26,87,90,6.20,CA,1595535186,Mo's Houseboat B&B,62.4588,-114.343
16,Tiksi,71.69,128.87,287.97,81,98,3.76,RU,1595534636,Arktika,71.6922,128.872
17,Ballina,-28.87,153.57,288.15,93,90,2.10,AU,1595534847,Ballina Manor Boutique Hotel,-28.8703,153.567
18,City of San Pedro,14.35,121.02,304.26,88,100,2.60,PH,1595535188,Palines Apartment and Guesthouse,14.3643,121.013


In [111]:
locations = hotel_df1[['Hotel_Lat','Hotel_Lng']]


hotel_layer = gmaps.marker_layer(locations)

fig = gmaps.figure(center=(30,0), zoom_level=1.2)
fig.add_layer(humidity_layer)
fig.add_layer(hotel_layer)
fig

Figure(layout=FigureLayout(height='420px'))

,City,Latitude,Longitude,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date,Hotel_Name
1,Bluff,-46.60,168.33,277.04,83,98,1.34,NZ,1595534626,
8,Yellowknife,62.46,-114.35,289.26,87,90,6.20,CA,1595535186,
16,Tiksi,71.69,128.87,287.97,81,98,3.76,RU,1595534636,
17,Ballina,-28.87,153.57,288.15,93,90,2.10,AU,1595534847,
18,City of San Pedro,14.35,121.02,304.26,88,100,2.60,PH,1595535188,
...,...,...,...,...,...,...,...,...,...,...
567,Tura,25.52,90.22,296.52,98,99,2.02,IN,1595535401,
568,Dolores,-36.31,-57.68,285.78,92,100,5.95,AR,1595535401,
569,Berlevåg,70.86,29.09,284.15,93,90,1.00,NO,1595535402,
571,Ol'ga,43.74,135.29,288.62,93,95,0.94,RU,1595535402,


In [113]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel_Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Latitude</dt><dd>{Latitude}</dd>
<dt>Longitude</dt><dd>{Longitude}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df1.iterrows()]
locations = hotel_df1[["Hotel_Lat", "Hotel_Lng"]]

In [114]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)

fig = gmaps.figure(center=(30,0), zoom_level=1.2)
fig.add_layer(humidity_layer)
fig.add_layer(hotel_layer)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))